# Reinforcement Learning Solution to the Towers of Hanoi Puzzle

_David Edwards_

In which I learn the benefits of better reading the requirements thoroughly...

Due to my being a single parent last week, I went through and did all of the printing, valid Moves, making and unmaking moves (which I ended up not using) and doing the myTupler, which takes a state and makes it into a tupler.

Point 1 of reading the requirements, whereas I needed to type `Q[(myTupler(state), move)]` if I had implemented the `stateMoveTuple(state, move)` as mentioned in the assignment, I could have used the more readable `Q[stateMoveTuple(state,move)]`

My next issue was using 1 (positive) for the goal state.  My original Q update line in `trainQ` looked like this:

`Q[(myTupler(stateOld), moveOld)] += rho * (Q[(myTupler(state), move)] - Q[(myTupler(stateOld), moveOld)])`

It actually kinda worked, and populated Q, but incorrectly.  The values all tended to be low numbers, trending towards zero.  After re-reading and seeing the -1, that was an easy change to make, but it broke everything.  After reading the Piazza post on what a proper Q dictionary would look like, I was able to tinker with that line until I realized I needed to include that -1 in the calculation.  A few more iterations, and I came up with this:

`Q[(myTupler(stateOld), moveOld)] += rho * (-1+Q[(myTupler(state), move)] - Q[(myTupler(stateOld), moveOld)])`

After that, everything just worked perfectly.

Once I had Q properly created, the testQ function was almost trivial.  I say _almost_ because I couldn't figure out where in Q we should start.  I spent an unfortunate amount of time thinking about that, before I realized that the start state should be the same as when training Q.  Once I figured that out, all I had to do was lookup the current state in Q, determine which was the largest value (closest to 0), and make that move, appending it to the list to return.

In [35]:
import copy as copy
import numpy as np
import random

def printState(state):
    '''
    Prints a Tower of Hanoi state.  Now, with added pegs.
    :param state: list of lists representing tower of hanoi state
    :return: prints out the state nicely.
    '''
    stateCopy = copy.deepcopy(state)
    maxLength = findLongest(stateCopy)
    for item in stateCopy:
        for x in range (len(item) , maxLength):
            item.insert(0, ' ')

    for i in range(maxLength):
        for j in range(len(stateCopy)):
            print("  {}".format(stateCopy[j][i]), end='')
        print()
    print('--|--|--|--\n\n')


def findLongest(state):
    '''
    Finds the longest (highest) peg in Towers of Hanoi.  Used to display in printState
    :param state: list of lists representing tower of hanoi state
    :return: length of longest item
    '''
    length = 0
    for item in state:
        length = max(length, len(item))

    return length


def validMoves(state):
    '''
    Returns a list of lists representing valid tower of hanoi moves from the given state.
    :param state: list of lists representing tower of hanoi state
    :return: list of lists representing valid tower of hanoi moves from the given state.
    '''
    validStates = []

    for itemToMove in state:
        # Check if the list is empty
        if itemToMove:
            # Iterate through the rest of the states
            for placeToMove in range(len(state)):
                # don't count as valid move if it's back to the same location
                if state.index(itemToMove) != placeToMove:
                    # valid move if the place to move is empty or the item to move is smaller
                    if len(state[placeToMove]) == 0 or itemToMove[0] < state[placeToMove][0]:
                        # append the tuple (move from, move to) to the valid states
                        validStates.append([state.index(itemToMove)+1, placeToMove+1])
    return validStates


def makeMove(state, move):
    '''
    Takes a move and makes it on a tower of hanoi state
    :param state: list of lists representing tower of hanoi state
    :param move: tuple representing a move from (peg1,peg2)
    :return:the state after the move was made
    '''
    item = state[move[0]-1].pop(0)
    state[move[1]-1].insert(0, item)
    return state


def unMakeMove(state, move):
    '''
    Reverses a move made by makeMove.  No longer used.
    :param state: list of lists representing tower of hanoi state
    :param move: move: tuple representing a move from (peg1,peg2)
    :return: the state after the move was made
    '''
    item = state[move[1]-1].pop(0)
    state[move[0]-1].insert(0, item)
    return state


def winner(state):
    '''
    Determines if a winning state occured
    :param state: list of lists representing tower of hanoi state
    :return: True if winning state, False otherwise.
    '''
    board = [[], [], [1,2,3]]
    return state == board


def myTupler(state):
    '''
    Need immutable type for key to dictionary
    :param state: list of lists representing tower of hanoi state
    :return: tuple representation of the state
    '''
    superTuple = tuple(tuple(s) for s in state)
    return superTuple


def epsilonGreedy(epsilon, Q, state, validMovesF):
    '''
    Makes either a random move, or tries the move which Q indicates is the best.
    :param epsilon: A decreasing number representing the level of randomness
    :param Q: Dictionary of state,move - value pairs, with the higher values being better moves
    :param state: list of lists representing tower of hanoi state
    :param validMovesF: function returning valid moves
    :return:
    '''
    goodMoves = validMovesF(state)
    if np.random.uniform() < epsilon:
        # Random Move
        return tuple(random.choice(goodMoves))
    else:
        # Greedy Move
        Qs = np.array([Q.get((myTupler(state),tuple(m)), 0.0) for m in goodMoves])
        return tuple(goodMoves[np.argmax(Qs)])


def trainQ(nRepetitions, learningRate, epsilonDecayFactor, validMovesF, makeMoveF):
    '''
    Creates and fills a dictionary, Q, representing the (state,move) - value pairs which, if followed
    should create the shortest path to the solution.
    :param nRepetitions: how many times to iterate through.  Higher numbers would generate more accurate results
    :param learningRate: How much to adjust the value part of the dictionary
    :param epsilonDecayFactor: how quickly to reduce the random factor.
    :param validMovesF: function returning valid moves of a state
    :param makeMoveF: function making a move on a state
    :return: the dictionary, Q, and a list containing the number of steps it took per iteration to find the goal state
    '''
    maxGames = nRepetitions
    rho = learningRate
    epsilonDecayRate = epsilonDecayFactor
    epsilon = 1.0
    Q = {}
    stepList = []
    # show the moves while debuggin
    showMoves = False

    for nGames in range(maxGames):
        # reduce the randomness every pass
        epsilon *= epsilonDecayRate
        step = 0
        # hardcoded start state
        state = [[1, 2, 3], [], []]
        done = False

        while not done:
            step += 1
            # grab either a random or best of the known moves
            move = epsilonGreedy(epsilon, Q, state, validMovesF)

            # we don't want to change state directly, and because state is a list of lists, need to do a
            # deepcopy on it, then make the move
            stateNew = copy.deepcopy(state)
            makeMoveF(stateNew, move)
            
            # if we haven't encountered this state,move combo, add it to Q
            if (myTupler(state), move) not in Q:
                Q[(myTupler(state), move)] = 0.0  # Initial Q value for new state, move
            
            # print if debugging
            if showMoves:
                printState(stateNew)
            if winner(stateNew):
                # We won!  backfill Q
                if showMoves:
                    print('End State, we won!')
                Q[(myTupler(state), move)] = -1.0
                done = True
                # we're keeping a list of the number of steps it took for each winning solution, so add it here.
                stepList.append(step)

            # update the Q which led us here using the learning factor, and the difference between the current state
            # and the old state
            if step > 1:
                Q[(myTupler(stateOld), moveOld)] += rho * (-1+Q[(myTupler(state), move)] - Q[(myTupler(stateOld), moveOld)])

            # Store the current state, move so we can access it for the next Q update
            stateOld, moveOld = state, move
            state = stateNew

    return Q,stepList


def testQ(Q, maxSteps, validMovesF, makeMoveF):
    '''
    Using the dictionary Q, and the initial state of the game, traverse and return the best path.
    :param Q: dictionary representing the (state,move) - value pairs which, if followed should create the shortest path to the solution.
    :param maxSteps: The number of steps to attempt before giving up.
    :param validMovesF: function returning valid moves of a state
    :param makeMoveF: function making a move on a state
    :return: list containing the states from start to finish
    '''
    state = [[1, 2, 3], [], []]
    statePath = []
    statePath.append(state)

    for i in range(maxSteps):
        if winner(state):
            return statePath
        goodMoves = validMovesF(state)
        Qs = np.array([Q.get((myTupler(state), tuple(m)), 0.0) for m in goodMoves])
        move = goodMoves[np.argmax(Qs)]
        nextState = copy.deepcopy(state)
        makeMoveF(nextState, move)
        statePath.append(nextState)
        state = nextState

    return "No path found"


In [36]:
%run -i A5grader.py


Testing validMoves([[1], [2], [3]])

--- 10/10 points. Correctly returned [[1, 2], [1, 3], [2, 3]]

Testing validMoves([[], [], [1, 2, 3]])

--- 10/10 points. Correctly returned [[3, 1], [3, 2]]

Testing makeMove([[], [], [1, 2, 3]], [3, 2])

--- 10/10 points. Correctly returned [[], [1], [2, 3]]

Testing makeMove([[2], [3], [1]], [1, 2])

--- 10/10 points. Correctly returned [[], [2, 3], [1]]

Testing   Q, steps = trainQ(1000, 0.5, 0.7, validMoves, makeMove).

--- 10/10 points. Q dictionary has correct number of entries.

--- 10/10 points. The mean of the number of steps is 7.59 which is correct.

Testing   path = testQ(Q, 20, validMoves, makeMove).

--- 20/20 points. Correctly returns path of length 8, less than 10.

A5 Execution Grade is 80/80

 Remaining 20 points will be based on your text describing the trainQ and test! functions.

A5 FINAL GRADE is __/100
